In [1]:
#!/usr/bin/env python3
import os
import re
import sys
import collections
import argparse
import tables
import itertools 
import scipy
import matplotlib
import glob
%matplotlib inline
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.sparse as sp_sparse
import scipy.io as sio
import matplotlib.mlab as mlab

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from multiprocessing import Pool
from collections import defaultdict
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.io as io

In [2]:
def outlier_plot(ax, fc_list, plot_x_val, plot_y_val, outlier_idx, plot_idx, color):
    outlier_fc = np.array([])
    outlier_y_val = np.array([])
    outlier_x_val = np.array([])
        
    idx = np.intersect1d(plot_idx, outlier_idx)
    for j in idx:
        if fc_list[j] > 1:
            outlier_fc = np.append(outlier_fc, get_fc_range(fc_list[j]))
        else:
            outlier_fc = np.append(outlier_fc, get_fc_range(1/fc_list[j]))
            
        outlier_x_val = np.append(outlier_x_val, plot_x_val[j])
        outlier_y_val = np.append(outlier_y_val, plot_y_val[j])
        
    ax.scatter(outlier_x_val, outlier_y_val,
               color=color,
               s=outlier_fc,
               marker='o',
               edgecolor='w')

def get_fc_range(val):
    if (val >= 4):
        fc_range = 200
    elif (val >= 2):
        fc_range = 100
    else:
        fc_range = 50
    return fc_range

In [3]:
length_list = [0, 248956422, 491149951, 689445510, 879660065, 1061198324,
               1232004303, 1391350276, 1536488912, 1674883629, 1808681051,
               1943767673, 2077042982, 2191407310, 2298451028, 2400442217,
               2490780562, 2574038003, 2654411288, 2713028904, 2777473071,
               2824183054, 2875001522, 3031042417]

chr_order = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
             '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', 'X', 'Y']

v2_FILE = '/project/GCRB/Hon_lab/shared/former_members/s160875/03.analysis/Mosaic-seq/CROP-DE-analysis_10X-66K_no_downsampling-CPM.hg38/\
combine_10sgRNAs-volcano/generate_annotations/plot_annotation.txt'

annot_df = pd.read_csv(v2_FILE,
        header = None,
        sep='\t',
        names = ['idx', 'gene_names', 'chromosome', 'pos', 'strand', 'color_idx', 'chr_idx'])

## Load global hit df with filter

In [4]:
global_df_columns = ['idx', 'gene_names', 'chromosome', 'pos', 'strand', 'color_idx', 
                     'chr_idx', 'region', 'num_cell', 'bin', 'pval', 'fc', 
                     'padj-Gaussian', 'fc_by_rand_dist_cpm']

In [5]:
global_df = pd.read_csv('./MB231-expressed_global_hit.csv')[global_df_columns]

In [6]:
global_df

,idx,gene_names,chromosome,pos,strand,color_idx,chr_idx,region,num_cell,bin,pval,fc,padj-Gaussian,fc_by_rand_dist_cpm
0,18084,HIST1H1C,chr6,1087254752,-,1,5,chr10:113069526-113070026,677,700,-6.028432,0.792954,-250.002469,0.763949
1,38927,GPHN,chr14,2257914717,+,1,13,chr10:113069526-113070026,677,700,-7.938176,0.628706,-86.263796,0.632332
2,6643,C2orf68,chr2,334568488,-,1,1,chr10:113069526-113070026,677,700,-5.500114,0.724300,-100.908600,0.718998
3,48820,MBP,chr18,2651171686,-,1,17,chr10:113069526-113070026,677,700,-5.630419,0.749883,-115.594203,0.761883
4,38206,AP1G2,chr14,2214975380,-,1,13,chr10:79113049-79113549,710,700,-5.106838,0.676728,-84.702626,0.681951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9424,958,EPB41,chr1,28887091,+,0,0,chr8:143948324-143948982,1170,1200,-3.817262,0.753423,-58.032234,0.742355
9425,42755,PARN,chr16,2415074945,-,1,15,chr8:143948324-143948982,1170,1200,-2.845566,0.729112,-77.332639,0.685506
9426,19796,ASF1A,chr6,1180092544,+,1,5,chr8:143948324-143948982,1170,1200,-2.143703,0.833946,-77.394754,0.774264
9427,36477,ZNF605,chr12,2076723979,-,1,11,chr8:143948324-143948982,1170,1200,-5.623257,0.642645,-61.840944,0.658351


In [11]:
for region in list(set(global_df['region'].values))[:10]:
    express_subset_df = global_df[global_df['region'] == region]

    up_idx = np.where(express_subset_df['fc_by_rand_dist_cpm'] > 1)[0]
    down_idx = np.where(express_subset_df['fc_by_rand_dist_cpm'] < 1)[0]
    plot_y_val = [0] * (len(express_subset_df['pval'].values))

    for i in up_idx:
        plot_y_val[i] = -express_subset_df['padj-Gaussian'].values[list(up_idx).index(i)]

    for i in down_idx:
        plot_y_val[i] = express_subset_df['padj-Gaussian'].values[list(down_idx).index(i)]

    plot_y_val=np.array(plot_y_val) 
    plot_x_val=express_subset_df['pos'].values
    plot_y_val[np.isinf(plot_y_val)] = 0

    odd_idx = np.where(express_subset_df.color_idx == 0)
    even_idx= np.where(express_subset_df.color_idx == 1)

    fc = express_subset_df['fc_by_rand_dist_cpm'].values


    num_sgrna_cell = express_subset_df['num_cell'].values[0]
    enh_chrom, left, right = re.split(r'[:-]+', region)
    up_cutoff = 1
    down_cutoff = 1


    import matplotlib.gridspec as gridspec

    fig = plt.figure(figsize=(14,8))
    gs = gridspec.GridSpec(nrows=1, ncols=11)

    #plot all genes
    ax0 = fig.add_subplot(gs[:, 0:9])

    ax0.scatter(plot_x_val[odd_idx],
               plot_y_val[odd_idx],
               s=1,
               color='#4d4d4d',
               marker='.')

    ax0.scatter(plot_x_val[even_idx],
               plot_y_val[even_idx],
               s=1,
               color='#e0e0e0',
               marker='.')

    ax0.set_title('%s (%d cells)'%(region, num_sgrna_cell),
                 fontsize=18)

    ax0.set_ylabel('p value adjusted',
                  fontsize = 15)

    #configurate the axis
    [ymin, ymax] = ax0.get_ylim()
    max_yval = max([np.absolute(ymin), np.absolute(ymax)])
    ax0.set_ylim([round(-max_yval-1),round(max_yval+1)])
    #ax0.set_ylim([-200, 200])
    ax0.set_xlim([-1e8, length_list[-1] + 1e8])
    [ymin, ymax] = ax0.get_ylim()
    ax0.tick_params(direction='in')
    [xmin, xmax] = ax0.get_xlim()

    #use absolute value for the y-axis
    corrected_ylabels = np.array([])
    labels = [np.absolute(int(i)) for i in ax0.get_yticks()]

    ax0.set_yticklabels(labels)

    #change the x-axis labels to chromosome names
    xtick_pos = np.array([])
    for i,e in enumerate(length_list):
        if i == 0:
            continue
        chrom_midpoint = (length_list[i-1] + e) / 2
        xtick_pos = np.append(xtick_pos, chrom_midpoint)

    print_ChrNames = np.array([])
    for i in chr_order:
        print_ChrNames = np.append(print_ChrNames, i[:1].upper() + i[1:])

    ax0.set_xticklabels(print_ChrNames, 
                        rotation='60',
                        va='top',
                        ha='center',
                        style='oblique',
                        family='monospace')

    for i,e in enumerate(length_list):
        if i == 0:
            continue
        if i % 2 == 0:
            ax0.fill_betweenx([ymin, ymax],
                              [length_list[i-1], length_list[i-1]],
                              [e, e],
                              color='#e0e0e0',
                              alpha=0.1)
        if i % 2 == 1:
            ax0.fill_betweenx([ymin, ymax],
                              [length_list[i-1], length_list[i-1]],
                              [e, e],
                              color='#4d4d4d',
                              alpha=0.1)
    #setup the grid
    #[s.set_visible(False) for s in ax0.spines.values()]
    ax0.yaxis.grid(linestyle = '--')
    ax0.set_xticks(xtick_pos)

    #plot a vertical line at the position of enhancer
    ax0.axvline(int(left)+length_list[chr_order.index(enh_chrom.strip('chr'))],
               color = '#7A68A6',
               ymin=ymin,
               ymax=ymax,
               linestyle='-.',
               alpha = 0.8)

    #plot the outliers
    outliers = []
    counter = 0
    for i,e in enumerate(plot_y_val):
        if (e < (-1* down_cutoff) or e > up_cutoff):
            outliers.append(counter)
        counter += 1

    if np.any(outliers):
        for j in outliers:
            if (plot_y_val[j] > 1) or (plot_y_val[j] < -1):
                gene_name = express_subset_df.iloc[j,1]
                gene_chr = express_subset_df.iloc[j,2]

                if plot_y_val[j] > 0:
                    ax0.text(plot_x_val[j] + (xmax*0.01), plot_y_val[j] + (ymax*0.01),
                             '%s'%(gene_name),
                             color = '#A60628',
                             fontsize=15)
                else:
                    ax0.text(plot_x_val[j] + (xmax*0.01), plot_y_val[j] + (ymax*0.01),
                             '%s'%(gene_name),
                             color='#348ABD',
                             fontsize=15)
        outlier_plot(ax0, fc, plot_x_val, plot_y_val, outliers, up_idx, '#A60628')
        outlier_plot(ax0, fc, plot_x_val, plot_y_val, outliers, down_idx, '#348ABD')


    #plot a legend for the circle size
    ax1 = fig.add_subplot(gs[:,10])

    y_len = ymax - ymin
    y_val = []
    for i in range(0,4):
        y_val.append(ymax - y_len * ((i + 1) * 0.05))

    size = [150, 80, 30, 1]
    legend_text = ['>=4-fold', '>=2-fold',
                   '<2-fold', 'not significant']
    for i,size in enumerate(size):
        ax1.scatter(0.5, y_val[i], 
                    color = 'k',
                    s=size,
                    marker='o')
        ax1.text(0.7, y_val[i],
                 '%s'%(legend_text[i]),
                 ha='left',
                 va='center',
                 fontsize = 12)

    ax1.axis('off')
    ax1.set_ylim([ymin, ymax])
    ax1.set_xlim([0.1,1])
    ax1.set_title("Mean CPM FC",
                  ha='left',
                  va='bottom',
                  fontsize = 15)


#    fig.savefig('/project/GCRB/Hon_lab/s426305/Analysis/Spade_test/MB231/MB231_SM_Manhattan/%s.dual_manhattan.ver2.png'%(region), dpi=500)

    plt.ioff()
    plt.clf()

/home2/s426305/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:70: UserWarning: FixedFormatter should only be used together with FixedLocator
/home2/s426305/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:89: UserWarning: FixedFormatter should only be used together with FixedLocator


<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

...

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>